In [1]:
import numpy as np
import pandas as pd
import os
os.chdir( r'E:\Work\CFZZ\Data\PreData2')

# 读取原始数据名称
def GetRawDataName(path = r'E:\Work\CFZZ\Data\PreData2'):
    os.chdir(path)
    name = os.listdir()
    return name[:4]

# 建立预处理后的数据名称，需要传入原始数据名称
# 原始数据为分页的xls文件，处理后数据名称
def RowToPreName(RawDataName = GetRawDataName(), WhichFormat = '.xlsx'):
    name = []
    for i in RawDataName:
        ss = i.split('.')[0] + WhichFormat
        name.append(ss)
    return name

# 条件筛选（按照营业部）客户
def SelectBySales(df):
    #df = pd.read_csv(data)
    df0 = df[(df['经营单元'] == '深圳深南大道(单元)')]
    return df0

#按“客户编号”列去重，方便后面的分组求和后的字符匹配
def  DeDuplication(df0):
    # 去重，为了后面匹配汉子字符
    df1 = df0.drop_duplicates(['客户编号'])
    # 把需要求和的、客户编号、时间删掉
    df1 = df1.drop(df1.ix[ : ,[ 0,-1,-2,-3,-4,-5,-6]], axis = 1)
    # 重新设置为文档
    df1 = df1.reset_index()
    # 把第一列删掉
    df1 = df1.drop(df1.ix[ : ,[0]], axis = 1)
    return df1

# 计算客户周期内的各种数据之和
def SumClientData(df0):
    df2 = df0.groupby('客户编号').agg(np.sum).reset_index()
    df2 = df2.drop(df2.ix[ : , ['月份','人员编号']], axis = 1)
    return df2

# 进行匹配并保存
def MatchClientData(df1, df2):
    df3 = pd.merge(df1, df2, how = 'left', on='客户编号')
    return df3

# 主程序部分
DataList = GetRawDataName(path = r'E:\Work\CFZZ\Data\PreData2')
NameList = RowToPreName(RawDataName = DataList, WhichFormat = '.xlsx')
for i in range(len(DataList)):
    df = pd.read_csv(DataList[i])  
    df0 = SelectBySales(df = df)
    df1 = DeDuplication(df0 = df0)
    df2 = SumClientData(df0 = df0)
    df3 = MatchClientData(df1 = df1, df2 = df2)
    name = NameList[i]
    df3.to_excel(name, index = None)

In [6]:
# 合并相加两个2016年的表
os.chdir( r'E:\Work\CFZZ\Data\WorkData2\Step2')
df = pd.read_excel('2016年.xlsx') 

df0 = SelectBySales(df = df)

df1 = df0.drop_duplicates(['客户编号'])
# 把需要求和的、客户编号、时间删掉
df1 = df1.drop(df1.ix[ : ,[-1,-2,-3,-4,-5,-6]], axis = 1)
# 重新设置为文档
df1 = df1.reset_index()
# 把第一列删掉
df1 = df1.drop(df1.ix[ : ,[0]], axis = 1)

df2 = df0.groupby('客户编号').agg(np.sum).reset_index()
df2 = df2.drop(df2.ix[ : , ['人员编号']], axis = 1)

ddf3 = pd.merge(df1, df2, how = 'outer', on='客户编号')
df3.to_excel('2016年汇总.xlsx', index = None)

In [8]:
# 计算2016均值
os.chdir( r'E:\Work\CFZZ\Data\WorkData2\Step2')
df = pd.read_excel('2016年汇总.xlsx') 
for i in range(7,13):
    df.ix[:,[i]] = df.ix[:,[i]]/12
df.to_excel('2016年月均.xlsx', index = None)

In [9]:
# 计算2017前5月均值
os.chdir( r'E:\Work\CFZZ\Data\WorkData2\Step2')
df = pd.read_excel('201701-201705存量.xlsx') 
for i in range(7,13):
    df.ix[:,[i]] = df.ix[:,[i]]/5
df.to_excel('2015年前5月均值.xlsx', index = None)

In [11]:
# 手动加和了佣金
# 匹配去年和今年的差别
os.chdir( r'E:\Work\CFZZ\Data\WorkData2\Step2')
df1 = pd.read_excel('2016年月均.xlsx') 
df2 = pd.read_excel('2015年前5月均值.xlsx')
df = pd.merge(df1,df2,how='outer',on='客户编号')
df.to_excel('深南存量比较.xlsx'， index = None)